# Nouns from Web List (not so good)

In [ ]:
import requests
from bs4 import BeautifulSoup

url="http://frequencylists.blogspot.com/2016/01/the-2980-most-frequently-used-german.html"

user_agent = 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
html = requests.get(url, headers={ "user-agent": user_agent }).text
print(html)

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html)

In [ ]:
nouns = {}
for span in soup.find("div", class_="post-body").find_all("span"):
    nounspan = span.select_one("b")
    if nounspan:
        singular = nounspan.text.split()[-1]
        if singular and len(singular) > 0:
            
            nounspan = span.select_one("i")
            if nounspan:
                plural = nounspan.text.split()[-1]
            nouns[singular] = singular
            if not nouns.get(plural):
                nouns[plural] = singular

In [ ]:
with open("nouns.txt", "w") as file:
    for n in noun_list:
        file.write(f"{n[0]}\n{n[1]}\n")

In [ ]:
nouns = list(nouns.keys()) + list(nouns.values())
nouns = set(nouns)

# Nouns from Wiktionary

https://github.com/gambolputty/german-nouns

In [3]:
import pandas as pd

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 5000)


In [4]:
df = pd.read_csv('nouns.csv', low_memory=False)
df.sample(10)

,lemma,pos,genus,genus 1,genus 2,genus 3,genus 4,nominativ singular,nominativ singular*,nominativ singular 1,nominativ singular 2,nominativ singular 3,nominativ singular 4,nominativ singular stark,nominativ singular schwach,nominativ singular gemischt,nominativ plural,nominativ plural*,nominativ plural 1,nominativ plural 2,nominativ plural 3,nominativ plural 4,nominativ plural stark,nominativ plural schwach,nominativ plural gemischt,genitiv singular,genitiv singular*,genitiv singular 1,genitiv singular 2,genitiv singular 3,genitiv singular 4,genitiv singular stark,genitiv singular schwach,genitiv singular gemischt,genitiv plural,genitiv plural*,genitiv plural 1,genitiv plural 2,genitiv plural 3,genitiv plural 4,genitiv plural stark,genitiv plural schwach,genitiv plural gemischt,dativ singular,dativ singular*,dativ singular 1,dativ singular 2,dativ singular 3,dativ singular 4,dativ singular stark,dativ singular schwach,dativ singular gemischt,dativ plural,dativ plural*,dativ plural 1,dativ plural 2,dativ plural 3,dativ plural 4,dativ plural stark,dativ plural schwach,dativ plural gemischt,akkusativ singular,akkusativ singular*,akkusativ singular 1,akkusativ singular 2,akkusativ singular 3,akkusativ singular 4,akkusativ singular stark,akkusativ singular schwach,akkusativ singular gemischt,akkusativ plural,akkusativ plural*,akkusativ plural 1,akkusativ plural 2,akkusativ plural 3,akkusativ plural 4,akkusativ plural stark,akkusativ plural schwach,akkusativ plural gemischt
36267,Kastell,Substantiv,n,NaN,NaN,NaN,NaN,Kastell,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kastelle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kastells,Kastelles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kastelle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kastell,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kastellen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kastell,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kastelle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48828,Nachtreisezug,Substantiv,m,NaN,NaN,NaN,NaN,Nachtreisezug,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nachtreisezüge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nachtreisezugs,Nachtreisezuges,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nachtreisezüge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nachtreisezug,Nachtreisezuge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nachtreisezügen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nachtreisezug,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nachtreisezüge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71558,Thum,"Toponym,Substantiv",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57340,Rambur,Substantiv,m,NaN,NaN,NaN,NaN,Rambur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rambure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ramburs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rambure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rambur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ramburen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rambur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rambure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48850,Nachtwächter,Substantiv,m,NaN,NaN,NaN,NaN,Nachtwächter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nachtwächter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nachtwächters,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nachtwächter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nachtwächter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nachtwächtern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nachtwächter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nachtwächter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53846,Pfeifkessel,Substantiv,m,NaN,NaN,NaN,NaN,Pfeifkessel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pfeifkessel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pfeifkessels,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pfeifkessel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pfeifkessel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pfeifkesseln,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pfeifkessel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pfeifkessel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18236,Elder Statesman,"Substantiv,Wortverbindung",m,NaN,NaN,NaN,NaN,Elder Statesman,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Elder Statesmen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Elder State

In [5]:
df['nominativ plural']

0           NaN
1        -anzen
2         -chen
3        -enzen
4           -er
          ...  
83285       NaN
83286       NaN
83287       NaN
83288       NaN
83289       NaN
Name: nominativ plural, Length: 83290, dtype: object

In [6]:
df[df['lemma'] == 'Verkauf']

,lemma,pos,genus,genus 1,genus 2,genus 3,genus 4,nominativ singular,nominativ singular*,nominativ singular 1,nominativ singular 2,nominativ singular 3,nominativ singular 4,nominativ singular stark,nominativ singular schwach,nominativ singular gemischt,nominativ plural,nominativ plural*,nominativ plural 1,nominativ plural 2,nominativ plural 3,nominativ plural 4,nominativ plural stark,nominativ plural schwach,nominativ plural gemischt,genitiv singular,genitiv singular*,genitiv singular 1,genitiv singular 2,genitiv singular 3,genitiv singular 4,genitiv singular stark,genitiv singular schwach,genitiv singular gemischt,genitiv plural,genitiv plural*,genitiv plural 1,genitiv plural 2,genitiv plural 3,genitiv plural 4,genitiv plural stark,genitiv plural schwach,genitiv plural gemischt,dativ singular,dativ singular*,dativ singular 1,dativ singular 2,dativ singular 3,dativ singular 4,dativ singular stark,dativ singular schwach,dativ singular gemischt,dativ plural,dativ plural*,dativ plural 1,dativ plural 2,dativ plural 3,dativ plural 4,dativ plural stark,dativ plural schwach,dativ plural gemischt,akkusativ singular,akkusativ singular*,akkusativ singular 1,akkusativ singular 2,akkusativ singular 3,akkusativ singular 4,akkusativ singular stark,akkusativ singular schwach,akkusativ singular gemischt,akkusativ plural,akkusativ plural*,akkusativ plural 1,akkusativ plural 2,akkusativ plural 3,akkusativ plural 4,akkusativ plural stark,akkusativ plural schwach,akkusativ plural gemischt
75153,Verkauf,Substantiv,m,NaN,NaN,NaN,NaN,Verkauf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Verkäufe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Verkaufes,Verkaufs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Verkäufe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Verkauf,Verkaufe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Verkäufen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Verkauf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Verkäufe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# for i, col in enumerate(df.columns):
#    print(i, col)
    
cols = df.columns[16:]
cols

Index(['nominativ plural', 'nominativ plural*', 'nominativ plural 1',
       'nominativ plural 2', 'nominativ plural 3', 'nominativ plural 4',
       'nominativ plural stark', 'nominativ plural schwach',
       'nominativ plural gemischt', 'genitiv singular', 'genitiv singular*',
       'genitiv singular 1', 'genitiv singular 2', 'genitiv singular 3',
       'genitiv singular 4', 'genitiv singular stark',
       'genitiv singular schwach', 'genitiv singular gemischt',
       'genitiv plural', 'genitiv plural*', 'genitiv plural 1',
       'genitiv plural 2', 'genitiv plural 3', 'genitiv plural 4',
       'genitiv plural stark', 'genitiv plural schwach',
       'genitiv plural gemischt', 'dativ singular', 'dativ singular*',
       'dativ singular 1', 'dativ singular 2', 'dativ singular 3',
       'dativ singular 4', 'dativ singular stark', 'dativ singular schwach',
       'dativ singular gemischt', 'dativ plural', 'dativ plural*',
       'dativ plural 1', 'dativ plural 2', 'dativ plural 

In [9]:
df[cols] = df[cols].fillna('')
df = df[df['pos'] == 'Substantiv']

In [10]:
import re

nouns = {}

for i, row in df.iterrows():
    lemma = row['lemma']
    
    # skip non-nouns, tokens with specials and acronyms
    if re.search(r'[,\-\s\.0-9]', lemma)!=None or re.search(r'[A-Z]', lemma[1:])!=None:
        continue
        
    nouns[lemma] = lemma

    # add flex forms
    for col in cols:
        flex = row[col]

        if flex != None and len(flex) > 1 and \
           re.search(r'[,\-\s\.0-9]', flex)==None and re.search(r'[A-Z]', flex[1:])==None:
            nouns[flex] = lemma
            
nouns

{'Aa': 'Aa',
 'Aas': 'Aas',
 'Aachener': 'Aachener',
 'Aacheners': 'Aachener',
 'Aachenern': 'Aachener',
 'Aachenerin': 'Aachenerin',
 'Aachenerinnen': 'Aachenerin',
 'Aad': 'Aad',
 'Aaden': 'Aad',
 'Aak': 'Aak',
 'Aake': 'Aake',
 'Aakes': 'Aak',
 'Aaks': 'Aak',
 'Aaken': 'Aake',
 'Aakerbeere': 'Aakerbeere',
 'Aakerbeeren': 'Aakerbeere',
 'Aal': 'Aal',
 'Aale': 'Aal',
 'Aals': 'Aal',
 'Aales': 'Aal',
 'Aalen': 'Aal',
 'Aalbaum': 'Aalbaum',
 'Aalbäume': 'Aalbaum',
 'Aalbaumes': 'Aalbaum',
 'Aalbaums': 'Aalbaum',
 'Aalbaume': 'Aalbaum',
 'Aalbäumen': 'Aalbaum',
 'Aalbeere': 'Aalbeere',
 'Aalbeeren': 'Aalbeere',
 'Aalbeerstrauch': 'Aalbeerstrauch',
 'Aalbeersträucher': 'Aalbeerstrauch',
 'Aalbeerstrauches': 'Aalbeerstrauch',
 'Aalbeerstrauche': 'Aalbeerstrauch',
 'Aalbeersträuchern': 'Aalbeerstrauch',
 'Aalbestand': 'Aalbestand',
 'Aalbestände': 'Aalbestand',
 'Aalbestandes': 'Aalbestand',
 'Aalbestands': 'Aalbestand',
 'Aalbestande': 'Aalbestand',
 'Aalbeständen': 'Aalbestand',
 'Aalbric

In [11]:
nouns['Haus']

'Haus'

In [12]:
# df = orig_df.copy()

In [17]:
df[df.lemma == 'All']

,lemma,pos,genus,genus 1,genus 2,genus 3,genus 4,nominativ singular,nominativ singular*,nominativ singular 1,nominativ singular 2,nominativ singular 3,nominativ singular 4,nominativ singular stark,nominativ singular schwach,nominativ singular gemischt,nominativ plural,nominativ plural*,nominativ plural 1,nominativ plural 2,nominativ plural 3,nominativ plural 4,nominativ plural stark,nominativ plural schwach,nominativ plural gemischt,genitiv singular,genitiv singular*,genitiv singular 1,genitiv singular 2,genitiv singular 3,genitiv singular 4,genitiv singular stark,genitiv singular schwach,genitiv singular gemischt,genitiv plural,genitiv plural*,genitiv plural 1,genitiv plural 2,genitiv plural 3,genitiv plural 4,genitiv plural stark,genitiv plural schwach,genitiv plural gemischt,dativ singular,dativ singular*,dativ singular 1,dativ singular 2,dativ singular 3,dativ singular 4,dativ singular stark,dativ singular schwach,dativ singular gemischt,dativ plural,dativ plural*,dativ plural 1,dativ plural 2,dativ plural 3,dativ plural 4,dativ plural stark,dativ plural schwach,dativ plural gemischt,akkusativ singular,akkusativ singular*,akkusativ singular 1,akkusativ singular 2,akkusativ singular 3,akkusativ singular 4,akkusativ singular stark,akkusativ singular schwach,akkusativ singular gemischt,akkusativ plural,akkusativ plural*,akkusativ plural 1,akkusativ plural 2,akkusativ plural 3,akkusativ plural 4,akkusativ plural stark,akkusativ plural schwach,akkusativ plural gemischt
2604,All,Substantiv,n,NaN,NaN,NaN,NaN,All,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,,,,,,,Alls,,,,,,,,,,,,,,,,,,All,,,,,,,,,,,,,,,,,,All,,,,,,,,,,,,,,,,,


In [16]:
df.pos.value_counts()

Substantiv    73416
Name: pos, dtype: int64

# Correction of spaCy's German Lemmas

In [14]:
from spacy.lang.de.lemmatizer import LOOKUP
import sys

In [15]:
# file = sys.stdout
file = open("lemmatizer_de.py", "w", encoding='utf-8')
file = open("lemmatizer_de_changes.py", "w", encoding='utf-8')

file.write("""# coding: utf8
from __future__ import unicode_literals

LOOKUP_DELTA = {\n""")

for word, lemma in LOOKUP.items():
    try:
        if word[0].isupper() and lemma.islower() and word.lower() not in LOOKUP and word in nouns:
            new_lemma = nouns[word]
            file.write(f'    "{word}": "{new_lemma}", # previous "{lemma}"\n')
            # print(f'    "{word}": "{new_lemma}" # previous "{lemma}"')
        else:
            pass
            # file.write(f'    "{word}": "{lemma}",\n')
        
    except Exception as e:
        print(e)
        print(f'    "{word}": "{lemma}"')
        break
            
file.write("}\n")
file.close()

In [ ]:
word = "Banden"
word[-2:]

In [5]:
"Banden"[:-1] in LOOKUP

True

In [3]:
"Bande" in LOOKUP

True

# Test

In [100]:
import spacy

nlp = spacy.load('de')

In [108]:
from IPython.core.display import display, HTML
from tabulate import tabulate

def print_nlp(doc, include_punct=False):
    """Print tokens with attributes for spaCy doc."""
    rows = []
    for token in doc:
        if not token.is_punct or include_punct:
            row = (token.text, token.lemma_, 
                   token.pos_, token.tag_, token.dep_,
                   token.is_punct, token.is_alpha, token.is_stop,
                   token.ent_type_, token.ent_iob_)
            rows.append(row)

    # generate HTML formatted table for display in Jupyter
    headers = ['text', 'lemma_', 'pos_', 'tag_', 'dep_', 
               'is_punct', 'is_alpha', 'is_stop', 'ent_type', 'ent_iob'] 
    display(HTML(tabulate(rows, headers=headers, tablefmt='html')))
    

In [152]:
from lemmatizer_de import LOOKUP as my_LOOKUP

nlp.vocab.morphology.lemmatizer.lookup_table = my_LOOKUP




[('tagger', <spacy.pipeline.pipes.Tagger at 0x1c66911fa58>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x1c66b3b86a8>),
 ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x1c66b3b8708>)]

In [155]:
LOOKUP['Gärtner']

'gärtnern'

In [167]:
def print_nlp(doc):
    print(" ".join([f"{t}/{t.lemma_}/{t.pos_}" for t in doc if not t.is_punct]))

texts = """Dieser Gärtner wohnt im Haus.
"""

for text in texts.split("\n"):
    doc = nlp(text)
    print_nlp(doc)

Dieser/Dieser/DET Gärtner/Gärtner/NOUN wohnt/wohnen/VERB im/im/ADP Haus/Haus/NOUN

